In [1]:
import math
import pandas as pd
import numpy as np
import time
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaForCausalLM, LlamaTokenizer

C:\Users\jo16726\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model = GPTNeoForCausalLM.from_pretrained("transformers/models--EleutherAI--gpt-neo-1.3B/snapshots/8282180b53cba30a1575e49de1530019e5931739", local_files_only=True)#.cuda()
tokenizer = GPT2Tokenizer.from_pretrained("transformers/models--EleutherAI--gpt-neo-1.3B/snapshots/8282180b53cba30a1575e49de1530019e5931739", local_files_only=True)
modelstr = "GPT-Neo-1.3B"

#accesstoken = ""
#model = LlamaForCausalLM.from_pretrained("transformers/model-llama2-7B", local_files_only=True).cuda()
#tokenizer = LlamaTokenizer.from_pretrained("transformers/model-llama2-7B", local_files_only=True)
#modelstr = "Llama-2 7B"

#accesstoken = ""
#model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-13b-hf", use_auth_token=accesstoken)
#tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf", use_auth_token=accesstoken)
#modelstr = "Llama-2 13B"

#model = GPTNeoXForCausalLM.from_pretrained("transformers/gpt-neox-20b", local_files_only=True)#.half().cuda
#tokenizer = GPTNeoXTokenizerFast.from_pretrained("transformers/gpt-neox-20b", local_files_only=True)

In [3]:
data_long = pd.read_csv('FIG-QA/Fig-QA Dataset copy.csv')

#data_negsample = pd.read_excel('FIG-QA/negsents results/negation sentences.xlsx')
#data_negsample_lem = pd.read_csv('FIG-QA/negsents results/negsents_lem.csv')

data = data_long
data = data[(data.labels!=-1)]
#dataname = "negsents_lem"

In [4]:
midphrase =  " In other words, " #" That is to say, "
sdata = data
scores1 = []
scores2 = []
classifications = []

for i in range(len(sdata[:])):
    # prepare prompt
    if sdata.iloc()[i]['startphrase'][-1] == ".":
        prompt1 = (sdata.iloc()[i]['startphrase'] + midphrase + sdata.iloc()[i]['ending1'])
    else:
        prompt1 = (sdata.iloc()[i]['startphrase'] + "." + midphrase + sdata.iloc()[i]['ending1'])
    if sdata.iloc()[i]['startphrase'][-1] == ".":
        prompt2 = (sdata.iloc()[i]['startphrase'] + midphrase + sdata.iloc()[i]['ending2'])
    else:
        prompt2 = (sdata.iloc()[i]['startphrase'] + "." + midphrase + sdata.iloc()[i]['ending2'])
        
    input_ids1 = tokenizer(prompt1, return_tensors="pt").input_ids#.to("cuda")
    loss1, logits1 = model(input_ids1, labels=input_ids1)[:2]
    input_ids2 = tokenizer(prompt2, return_tensors="pt").input_ids#.to("cuda")
    loss2, logits2 = model(input_ids2, labels=input_ids2)[:2]
    Pxy1 = math.exp(-1.0 * loss1)
    Pxy2 = math.exp(-1.0 * loss2)
    P1 = Pxy1/(Pxy1+Pxy2)
    P2 = Pxy2/(Pxy1+Pxy2)
    scores1.append(P1)
    scores2.append(P2)
    if sdata.iloc()[i]['labels'] == 0:
        if  P1 > P2: 
            classification = 1
        else:
            classification = 0
    elif sdata.iloc()[i]['labels'] == 1:
        if  P2 > P1: 
            classification = 1
        else:
            classification = 0
    else:
        classification = ""
    classifications.append(classification)
    
    if i%256 == 0:
        print(i,"/",len(sdata))
        
sdata["{} 1".format(modelstr)], sdata["{} 2".format(modelstr)], sdata["{} bin".format(modelstr)] = scores1, scores2, classifications

print(np.mean(classifications))
sdata.to_csv("FIG-QA/test_m2_GPT Neo.csv", index=False)

0 / 9110
256 / 9110
512 / 9110
768 / 9110
1024 / 9110
1280 / 9110
1536 / 9110
1792 / 9110
2048 / 9110
2304 / 9110
2560 / 9110
2816 / 9110
3072 / 9110
3328 / 9110
3584 / 9110
3840 / 9110
4096 / 9110
4352 / 9110
4608 / 9110
4864 / 9110
5120 / 9110
5376 / 9110
5632 / 9110
5888 / 9110
6144 / 9110
6400 / 9110
6656 / 9110
6912 / 9110
7168 / 9110
7424 / 9110
7680 / 9110
7936 / 9110
8192 / 9110
8448 / 9110
8704 / 9110
8960 / 9110
0.5802414928649835


C:\Users\jo16726\AppData\Local\Temp\ipykernel_24088\646748240.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr)], sdata["{} 2".format(modelstr)], sdata["{} bin".format(modelstr)] = scores1, scores2, classifications
C:\Users\jo16726\AppData\Local\Temp\ipykernel_24088\646748240.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata["{} 1".format(modelstr)], sdata["{} 2".format(modelstr)], sdata["{} bin".format(modelstr)] = scores1, scores2, classifications
C:\Users\jo16726